In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer  # null 값 제거 시
from sklearn.preprocessing import PolynomialFeatures  # 피쳐 계산해 새로운 피쳐 만들기
from sklearn.preprocessing import StandardScaler # 
from sklearn.feature_selection import VarianceThreshold # feature importance 계산할 때 분산 적은 것 걸러내려고?
from sklearn.feature_selection import SelectFromModel  # feature importance 계산해 중요 피쳐 뽑아내려고?
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [30]:
import glob
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [31]:
file_path = glob.glob('./data/*.csv')

In [32]:
file_path

['./data\\sample_submission.csv', './data\\test.csv', './data\\train.csv']

In [33]:
DEBUG = True

In [34]:
if DEBUG:
    NROWS = 1000
else:
    NROWS = None

In [113]:
%%time
train = pd.read_csv(file_path[2], nrows=NROWS)
test = pd.read_csv(file_path[1], nrows=NROWS)

Wall time: 911 ms


In [111]:
%%time
train = pd.read_csv(file_path[2], nrows=None)
test = pd.read_csv(file_path[1], nrows=None)

Wall time: 15.3 s


In [118]:
%%time
# random sampling
train = train.sample(frac=0.1)
test = test.sample(frac=0.1)

Wall time: 39 ms


In [119]:
train.shape, test.shape

((5000, 59), (5000, 58))

---

In [120]:
# imbalance 한 것 생각 시
from sklearn.model_selection import StratifiedKFold

In [121]:
fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1980)

In [122]:
for trn_idx, val_idx in fold.split(train, train['target']):
    break

In [123]:
trn_idx

array([   0,    1,    2, ..., 4996, 4997, 4998])

In [124]:
val_idx

array([   4,   17,   22,   23,   31,   46,   50,   53,   79,   86,   95,
        100,  108,  116,  124,  126,  129,  144,  157,  167,  190,  226,
        236,  240,  245,  283,  284,  288,  290,  329,  332,  334,  345,
        346,  351,  381,  391,  403,  407,  422,  425,  431,  432,  485,
        526,  536,  541,  556,  563,  570,  577,  585,  589,  595,  605,
        634,  635,  639,  647,  655,  660,  686,  687,  698,  702,  711,
        714,  742,  756,  758,  760,  761,  765,  769,  780,  788,  792,
        811,  820,  825,  831,  834,  842,  858,  878,  888,  903,  910,
        938,  942,  950,  964,  979,  990,  992,  998, 1000, 1005, 1009,
       1021, 1034, 1041, 1085, 1112, 1115, 1116, 1124, 1129, 1130, 1144,
       1162, 1165, 1167, 1181, 1183, 1196, 1214, 1232, 1248, 1250, 1252,
       1254, 1265, 1270, 1286, 1303, 1308, 1316, 1336, 1353, 1356, 1362,
       1374, 1384, 1391, 1392, 1399, 1432, 1436, 1440, 1454, 1460, 1479,
       1484, 1485, 1487, 1498, 1500, 1504, 1507, 15

In [125]:
trn = train.iloc[trn_idx]

In [126]:
val = train.iloc[val_idx]

---

In [127]:
DEBUG = True

In [128]:
if DEBUG:
    NROWS = 50000
else:
    NROWS = None

In [129]:
%%time
train = pd.read_csv(file_path[2], nrows=NROWS)
test = pd.read_csv(file_path[1], nrows=NROWS)
# random sampling
train = train.sample(frac=0.1)
test = test.sample(frac=0.1)

Wall time: 937 ms


In [130]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
33914,85084,0,0,1,3,0,4,0,1,0,0,0,0,0,0,0,4,0,0,1,0.1,0.1,-1.000000,7,1,0,2,0,1,1,1,2,1,1,0,0.374166,0.857285,0.336155,3.464102,0.9,0.8,0.1,0,0,4,7,10,3,12,5,0,2,17,0,1,0,0,0,0
6412,16391,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,6,1,0,0,0.5,0.2,0.612883,11,1,-1,0,-1,1,1,1,2,1,65,3,0.316228,0.583166,0.350714,2.828427,0.2,0.8,0.2,3,2,9,3,6,2,7,6,2,2,10,1,1,1,0,1,0
1626,4165,0,2,1,6,1,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.6,0.4,0.960143,10,1,-1,0,1,14,1,1,0,1,104,1,0.316070,0.438018,0.301662,0.000000,0.2,0.9,0.8,3,3,8,2,10,3,7,7,2,4,10,0,0,0,0,0,0
20034,50207,0,0,1,8,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0.3,0.2,-1.000000,11,0,-1,0,-1,11,1,1,2,1,3,2,0.424264,1.098684,0.434971,3.464102,0.5,0.8,0.7,2,3,7,3,7,1,5,5,2,3,7,1,1,1,1,0,0
5339,13635,0,0,2,0,0,-1,0,0,0,1,0,0,0,0,0,7,0,1,0,0.9,0.4,0.460977,7,1,1,1,0,14,-1,0,1,1,8,3,0.400000,0.991659,-1.000000,3.741657,0.7,0.1,0.2,3,3,9,3,9,2,3,12,1,6,12,0,1,1,0,1,0


In [131]:
cat_cols = [col for col in train.columns if 'cat' in col]

In [132]:
cat_cols

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [133]:
train[cat_cols[13]].value_counts()

104    713
103    209
64     195
87     146
28     113
      ... 
81      11
63      11
59       9
75       9
56       7
Name: ps_car_11_cat, Length: 104, dtype: int64

In [134]:
for col in cat_cols:
    print(col, train[col].nunique())

ps_ind_02_cat 5
ps_ind_04_cat 3
ps_ind_05_cat 8
ps_car_01_cat 12
ps_car_02_cat 2
ps_car_03_cat 3
ps_car_04_cat 8
ps_car_05_cat 3
ps_car_06_cat 18
ps_car_07_cat 3
ps_car_08_cat 2
ps_car_09_cat 6
ps_car_10_cat 3
ps_car_11_cat 104


high cardinality category : ps_car_11_cat 104개

In [135]:
train.shape

(5000, 59)

In [136]:
train.drop_duplicates()
train.shape

(5000, 59)

In [137]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 33914 to 15704
Data columns (total 59 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              5000 non-null   int64  
 1   target          5000 non-null   int64  
 2   ps_ind_01       5000 non-null   int64  
 3   ps_ind_02_cat   5000 non-null   int64  
 4   ps_ind_03       5000 non-null   int64  
 5   ps_ind_04_cat   5000 non-null   int64  
 6   ps_ind_05_cat   5000 non-null   int64  
 7   ps_ind_06_bin   5000 non-null   int64  
 8   ps_ind_07_bin   5000 non-null   int64  
 9   ps_ind_08_bin   5000 non-null   int64  
 10  ps_ind_09_bin   5000 non-null   int64  
 11  ps_ind_10_bin   5000 non-null   int64  
 12  ps_ind_11_bin   5000 non-null   int64  
 13  ps_ind_12_bin   5000 non-null   int64  
 14  ps_ind_13_bin   5000 non-null   int64  
 15  ps_ind_14       5000 non-null   int64  
 16  ps_ind_15       5000 non-null   int64  
 17  ps_ind_16_bin   5000 non-nul

obj 는 string 인데 이게 없으니 라벨 인코딩 안써도 된다.

+ Metadata

- store meta information in a data frame
- helpful in select variables
- role
- level
- keep
- dtype

In [138]:
data = []

for f in train.columns:
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
    
    #defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    else:
        level = 'ordinal'
    '''
    elif train[f].dtype == int:
        level = 'ordinal'
    '''
    keep = True
    if f == 'id':
        keep = False
    
    dtype = train[f].dtype
    
    f_dict = {
        'varname' : f,
        'role' : role,
        'level' : level,
        'keep' : keep,
        'dtype' : dtype
    }
    data.append(f_dict)

meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [139]:
meta

,role,level,keep,dtype
varname,,,,
id,id,nominal,False,int64
target,target,binary,True,int64
ps_ind_01,input,ordinal,True,int64
ps_ind_02_cat,input,nominal,True,int64
ps_ind_03,input,ordinal,True,int64
ps_ind_04_cat,input,nominal,True,int64
ps_ind_05_cat,input,nominal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


In [140]:
train['ps_car_11'].dtype

dtype('int64')

In [141]:
train['ps_car_11'].dtype == int

False

In [142]:
train['ps_car_12'].dtype

dtype('float64')

In [143]:
train['ps_car_12'].dtype == float

True

In [144]:
train['ps_car_12'].dtype == float

True

In [145]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Index(['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat',
       'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat',
       'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat',
       'ps_car_10_cat', 'ps_car_11_cat'],
      dtype='object', name='varname')

In [146]:
pd.DataFrame({'count': meta.groupby(['role', 'level'])['role'].size()}).reset_index()

,role,level,count
0,id,nominal,1
1,input,binary,17
2,input,interval,10
3,input,nominal,14
4,input,ordinal,16
5,target,binary,1


### Descriptive statistics

+ we can also apply the `describe` method on the dataframe. However, it does not make sense to calculate the mean, std, ... on categorical variables and the id variable.

+ we will explore the categorical variables visually later.

+ thanks to our meta file we can easily select the variables on which we want to compute the descriptive statistics.

+ **Interval variables**

In [147]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.606480,0.446120,0.546124,0.381473,0.815231,0.273805,3.061510,0.447300,0.452960,0.451600
std,0.288982,0.413806,0.808826,0.059244,0.227825,0.361899,0.740045,0.289388,0.286167,0.288926
min,0.000000,0.000000,-1.000000,0.315278,0.324634,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.200000,0.517657,0.316228,0.669834,0.330757,2.828427,0.200000,0.200000,0.200000
50%,0.700000,0.300000,0.724137,0.374166,0.769492,0.368782,3.316625,0.400000,0.500000,0.500000
75%,0.900000,0.600000,1.010028,0.408412,0.910283,0.396485,3.605551,0.700000,0.700000,0.700000
max,0.900000,1.800000,2.525124,1.264911,2.702314,0.568331,3.741657,0.900000,0.900000,0.900000


In [148]:
v

Index(['ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_12', 'ps_car_13',
       'ps_car_14', 'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03'],
      dtype='object', name='varname')

+ feature selection 할 때 쓰냐 안쓰냐 업데이트 하는게 중요하더라

+ **reg variables**
+ only ps_reg_03 has missing values.
+ the range differs between the variables. we could apply scaling(StandardScaler), but it depends on the classifier we will want to use. ex)tree 모델은 스케일링 영향 x

+ **car variables**
+ ps_car_12 and ps_car_15 have missing values
+ the range differs and we could apply scailing.

+ **calc variables**
+ no missing values
+ this seems to be some kind of ratio as the maximum is 0.9
+ all three `_calc` varialbes have very similar distributions

we can see that the range of the interval variables is rather small. Perhaps some transformation is already applied in order to anonymize the data?

### Ordinal variables

In [149]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

,ps_ind_01,ps_ind_03,ps_ind_14,ps_ind_15,ps_car_11,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,1.928600,4.413200,0.012800,7.318000,2.337400,2.350600,1.899600,7.703600,3.021200,9.236200,2.353000,8.396800,5.469200,1.459000,2.858800,7.534400
std,1.993363,2.701986,0.140855,3.540139,0.838631,1.133639,1.141131,1.332705,1.393965,1.461519,1.260281,2.874057,2.368158,1.208557,1.677088,2.763683
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,5.000000,2.000000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000
50%,1.000000,4.000000,0.000000,7.000000,3.000000,2.000000,2.000000,8.000000,3.000000,9.000000,2.000000,8.000000,5.000000,1.000000,3.000000,7.000000
75%,3.000000,6.000000,0.000000,10.000000,3.000000,3.000000,3.000000,9.000000,4.000000,10.000000,3.000000,10.000000,7.000000,2.000000,4.000000,9.000000
max,7.000000,11.000000,3.000000,13.000000,3.000000,5.000000,6.000000,10.000000,8.000000,12.000000,6.000000,21.000000,17.000000,8.000000,11.000000,22.000000


+ only one missing variable : ps_car_11
+ we could apply scailing to deal with the different ranges

### Binary variables

In [150]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

,target,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.037400,0.396400,0.256400,0.169200,0.178000,0.000400,0.002600,0.00900,0.000800,0.662400,0.117000,0.150200,0.12260,0.62780,0.567400,0.289800,0.344800,0.147200
std,0.189759,0.489198,0.436689,0.374966,0.382551,0.019998,0.050929,0.09445,0.028276,0.472939,0.321452,0.357303,0.32801,0.48344,0.495486,0.453715,0.475351,0.354341
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.00000,1.00000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.00000,1.00000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000


+ a prior in the train data is 3.645% which is **strongly imbalanced**.
+ from the means we can conclude that for most variables the value is zero in most cases.

### Handling imbalanced classes

+ as we mentioned above the proportion of records with target=1 is far less than target=0. 
+ This can lead to a model that has great accuracy but does have any added value in practice. 
+ Two possible strategies to deal with this problem are:

```   
+  oversampling records with target=1
+  undersampling records with target=0
```   
+ There are many more strategies of course and MachineLearningMastery.com gives a nice overview. 
+ As we have a rather large training set, we can go for undersampling.
+ 오버샘플링 좋았던 적이 별로 없었다 함.
+ 샘플링 패키지 imbalanced-learn

In [151]:
desired_apriori = 0.10

# get the indices per target value
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

# get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# calculate the undersampling rate and resulting number of records with target=0
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print(f'Rate to undersample records with target=0: {undersampling_rate}')
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

# return undersampled data frame
train = train.loc[idx_list].reset_index(drop=True)

Rate to undersample records with target=0: 0.3496779555370871
Number of records with target=0 after undersampling: 1683


### Data quality check

#### Checking missing values
+ missings are represented as -1

In [152]:
vars_with_missing = []

for f in train.columns:
    missings = train.loc[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print(f'Variable {f} has {missings} reocords ({missings_perc:.2%}) with missing values')
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))    

Variable ps_ind_02_cat has 1 reocords (0.05%) with missing values
Variable ps_ind_04_cat has 1 reocords (0.05%) with missing values
Variable ps_ind_05_cat has 12 reocords (0.64%) with missing values
Variable ps_reg_03 has 345 reocords (18.45%) with missing values
Variable ps_car_03_cat has 1275 reocords (68.18%) with missing values
Variable ps_car_05_cat has 795 reocords (42.51%) with missing values
Variable ps_car_07_cat has 26 reocords (1.39%) with missing values
Variable ps_car_14 has 142 reocords (7.59%) with missing values
In total, there are 8 variables with missing values


+ ps_car_03_cat and ps_car_05_cat have a large proportion of records with missing values. Remove these variables.
+ 맘대로 지우면 안되는데.


In [153]:
train[['ps_car_03_cat', 'target']].groupby('ps_car_03_cat').mean()

,target
ps_car_03_cat,
-1,0.092549
0,0.088889
1,0.132432


In [154]:
train[['ps_car_05_cat', 'target']].groupby('ps_car_05_cat').mean()

,target
ps_car_05_cat,
-1,0.096855
0,0.090196
1,0.113274


+ missing value 자체가 target에 유의미할 수가 있는데 여기서는 별 의미 없는 것으로 보인다.

+ For the other categorical variables with missing values, we can leave the missing value -1 as such.

+ ps_reg_03 (continuous) has missing values for 18% of all records. Replace by the mean.

+ ps_car_11 (ordinal) has only 5 records with misisng values. Replace by the mode.

+ ps_car_12 (continuous) has only 1 records with missing value. Replace by the mean.
+ ps_car_14 (continuous) has missing values for 7% of all records. Replace by the mean.

missing value 도 함부로 채우면 안된다. 
미싱밸류 없는 변수로 학습하고 채우는 방법도 있다.
바이어스 줄 수 있으니 함부로 채우지 말자

In [155]:
# dropping the variables with too many missing values
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, axis=1, inplace=True)
meta.loc[(vars_to_drop), 'keep'] = False # updating the meta


In [158]:
# missing value 채우기 groupby 로 하기 ps_reg_03 채우고 싶은 경우
temp_series = train[['ps_car_01_cat', 'ps_car_02_cat', 'ps_reg_03']].groupby(['ps_car_01_cat', 'ps_car_02_cat']).mean()
temp_series

ps_reg_03
ps_car_01_cat ps_car_02_cat           
0             0               0.785053
              1               0.536584
1             1               0.977097
2             0               1.253994
              1               0.464361
3             0               0.675488
              1               0.552798
4             0              -0.635267
              1               0.205724
5             0               0.310521
              1               0.063997
6             0               0.888664
              1               0.440862
7             0               0.437962
              1               0.309102
8             0               0.888370
              1               0.187927
9             0               0.540256
              1               0.720827
10            0               0.878982
              1               0.715319
11            0               1.070394
              1               0.729536

In [159]:
temp_series.reset_index(inplace=True)
temp_series

,ps_car_01_cat,ps_car_02_cat,ps_reg_03
0,0,0,0.785053
1,0,1,0.536584
2,1,1,0.977097
3,2,0,1.253994
4,2,1,0.464361
5,3,0,0.675488
6,3,1,0.552798
7,4,0,-0.635267
8,4,1,0.205724
9,5,0,0.310521


In [161]:
train.loc[train['ps_reg_03'] == -1]

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_04_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,38112,0,0,1,2,1,0,0,0,1,0,0,0,0,0,0,10,0,0,0,0.1,0.2,-1.0,5,1,0,1,1,1,0,1,70,3,0.424264,0.874454,0.342929,3.464102,0.6,0.1,0.7,1,2,7,2,8,1,15,11,0,4,4,0,1,0,0,0,0
2,118542,0,0,1,5,1,0,0,1,0,0,0,0,0,0,0,8,1,0,0,0.9,0.8,-1.0,10,1,0,4,1,0,0,1,84,2,0.316228,0.842112,0.366742,3.741657,0.4,0.1,0.6,2,0,6,0,8,2,18,4,0,4,10,0,1,1,1,1,0
6,22863,0,3,1,10,0,0,0,0,0,1,0,0,0,0,0,13,0,1,0,0.1,0.3,-1.0,7,1,0,11,1,1,0,1,99,2,0.316228,0.559866,0.368782,2.236068,0.2,0.3,0.0,3,3,8,1,7,1,8,3,2,1,11,0,0,0,1,0,0
15,100388,0,2,2,2,1,0,0,1,0,0,0,0,0,0,0,10,0,0,0,0.3,0.1,-1.0,7,1,0,14,1,1,2,1,60,1,0.316228,0.638309,0.307083,3.316625,0.4,0.1,0.8,4,1,10,0,12,5,6,5,3,1,6,0,1,0,0,0,1
20,41499,0,0,1,3,0,0,1,0,0,0,0,0,0,0,0,4,0,0,1,0.9,0.1,-1.0,4,1,0,1,1,1,0,1,31,3,0.374166,0.685520,0.385487,2.645751,0.0,0.8,0.9,5,1,10,3,9,2,3,7,2,4,6,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844,61733,1,1,1,4,1,6,1,0,0,0,0,0,0,0,0,13,1,0,0,0.3,0.2,-1.0,7,1,0,1,1,1,2,1,65,1,0.316228,0.524542,0.350714,2.449490,0.4,0.7,0.0,3,2,8,4,10,3,9,8,0,5,8,0,1,0,0,0,1
1856,26845,1,0,2,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,1,0.3,0.0,-1.0,11,1,0,0,1,1,0,1,86,3,0.400000,1.014329,-1.000000,3.605551,0.4,0.5,0.0,1,2,9,5,9,2,3,5,3,1,12,0,0,1,1,1,0
1860,112706,1,7,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.4,0.0,-1.0,4,1,8,14,1,0,0,1,18,2,0.565685,1.708299,0.565685,3.741657,0.5,0.6,0.6,3,3,8,3,9,3,7,4,2,2,7,0,1,1,0,0,0
1862,68824,1,0,1,6,0,0,0,1,0,0,0,0,0,0,0,9,1,0,0,0.1,0.2,-1.0,10,0,9,15,1,1,0,1,41,2,0.547723,1.263023,0.532917,2.645751,0.1,0.1,0.3,2,3,9,4,10,3,8,4,2,1,10,0,0,1,1,0,0


In [163]:
train.loc[train['ps_reg_03'] == -1].merge(temp_series, on=['ps_car_01_cat', 'ps_car_02_cat'], how='left', suffixes=('_old', '_new'))

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03_old,ps_car_01_cat,ps_car_02_cat,ps_car_04_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_reg_03_new
0,38112,0,0,1,2,1,0,0,0,1,0,0,0,0,0,0,10,0,0,0,0.1,0.2,-1.0,5,1,0,1,1,1,0,1,70,3,0.424264,0.874454,0.342929,3.464102,0.6,0.1,0.7,1,2,7,2,8,1,15,11,0,4,4,0,1,0,0,0,0,0.063997
1,118542,0,0,1,5,1,0,0,1,0,0,0,0,0,0,0,8,1,0,0,0.9,0.8,-1.0,10,1,0,4,1,0,0,1,84,2,0.316228,0.842112,0.366742,3.741657,0.4,0.1,0.6,2,0,6,0,8,2,18,4,0,4,10,0,1,1,1,1,0,0.715319
2,22863,0,3,1,10,0,0,0,0,0,1,0,0,0,0,0,13,0,1,0,0.1,0.3,-1.0,7,1,0,11,1,1,0,1,99,2,0.316228,0.559866,0.368782,2.236068,0.2,0.3,0.0,3,3,8,1,7,1,8,3,2,1,11,0,0,0,1,0,0,0.309102
3,100388,0,2,2,2,1,0,0,1,0,0,0,0,0,0,0,10,0,0,0,0.3,0.1,-1.0,7,1,0,14,1,1,2,1,60,1,0.316228,0.638309,0.307083,3.316625,0.4,0.1,0.8,4,1,10,0,12,5,6,5,3,1,6,0,1,0,0,0,1,0.309102
4,41499,0,0,1,3,0,0,1,0,0,0,0,0,0,0,0,4,0,0,1,0.9,0.1,-1.0,4,1,0,1,1,1,0,1,31,3,0.374166,0.685520,0.385487,2.645751,0.0,0.8,0.9,5,1,10,3,9,2,3,7,2,4,6,0,0,1,1,0,0,0.205724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,61733,1,1,1,4,1,6,1,0,0,0,0,0,0,0,0,13,1,0,0,0.3,0.2,-1.0,7,1,0,1,1,1,2,1,65,1,0.316228,0.524542,0.350714,2.449490,0.4,0.7,0.0,3,2,8,4,10,3,9,8,0,5,8,0,1,0,0,0,1,0.309102
341,26845,1,0,2,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,1,0.3,0.0,-1.0,11,1,0,0,1,1,0,1,86,3,0.400000,1.014329,-1.000000,3.605551,0.4,0.5,0.0,1,2,9,5,9,2,3,5,3,1,12,0,0,1,1,1,0,0.729536
342,112706,1,7,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.4,0.0,-1.0,4,1,8,14,1,0,0,1,18,2,0.565685,1.708299,0.565685,3.741657,0.5,0.6,0.6,3,3,8,3,9,3,7,4,2,2,7,0,1,1,0,0,0,0.205724
343,68824,1,0,1,6,0,0,0,1,0,0,0,0,0,0,0,9,1,0,0,0.1,0.2,-1.0,10,0,9,15,1,1,0,1,41,2,0.547723,1.263023,0.532917,2.645751,0.1,0.1,0.3,2,3,9,4,10,3,8,4,2,1,10,0,0,1,1,0,0,0.878982


+ 이렇게 groupby 써서 세분류로 들어가 각기 채울 수 있다.
---

In [165]:
mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')

In [167]:
mean_imp.fit_transform(train[['ps_reg_03']])

array([[0.903807  ],
       [0.92533778],
       [0.903807  ],
       ...,
       [1.19608737],
       [1.89835455],
       [0.903807  ]])

In [168]:
mean_imp.fit_transform(train[['ps_reg_03']]).shape

(1870, 1)

In [169]:
# 2차원 벡터가 되니까 풀어준다. ravel()
mean_imp.fit_transform(train[['ps_reg_03']]).ravel().shape

(1870,)

In [170]:
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

### checking the cardinality of the categorical variables
+ cardinality refers to the number of different values in a variable.
+ as we will create dummy variables from the categorical variables later on, we need to check whether there are vairables with many distinct values.
+ we should handle these vairables differently as they would result in many dummy variables.

In [172]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Varialbe {} has {} distinct values'.format(f, dist_values))

Varialbe ps_ind_02_cat has 5 distinct values
Varialbe ps_ind_04_cat has 3 distinct values
Varialbe ps_ind_05_cat has 8 distinct values
Varialbe ps_car_01_cat has 12 distinct values
Varialbe ps_car_02_cat has 2 distinct values
Varialbe ps_car_04_cat has 8 distinct values
Varialbe ps_car_06_cat has 18 distinct values
Varialbe ps_car_07_cat has 3 distinct values
Varialbe ps_car_08_cat has 2 distinct values
Varialbe ps_car_09_cat has 5 distinct values
Varialbe ps_car_10_cat has 3 distinct values
Varialbe ps_car_11_cat has 104 distinct values


In [174]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

sum = 0
for f in v:
    dist_values = train[f].nunique()
    sum += dist_values
    print('Varialbe {} has {} distinct values'.format(f, dist_values))

Varialbe ps_ind_02_cat has 5 distinct values
Varialbe ps_ind_04_cat has 3 distinct values
Varialbe ps_ind_05_cat has 8 distinct values
Varialbe ps_car_01_cat has 12 distinct values
Varialbe ps_car_02_cat has 2 distinct values
Varialbe ps_car_04_cat has 8 distinct values
Varialbe ps_car_06_cat has 18 distinct values
Varialbe ps_car_07_cat has 3 distinct values
Varialbe ps_car_08_cat has 2 distinct values
Varialbe ps_car_09_cat has 5 distinct values
Varialbe ps_car_10_cat has 3 distinct values
Varialbe ps_car_11_cat has 104 distinct values


In [175]:
sum

173

In [176]:
sum - 104

69

+ column 이 늘어나면 연산량이 늘어남
+ tree 모델은 스무고개 같은 건데 column이 많으면 물어볼 질문이 많아진다. 중요한 질문을 못 건드릴 가능성도 있다. 답으로 가기 위한 과정이 길어진다. 원하지도 않은 split이 생길 가능성도 있다.

+ **mean encoding**
  + 타겟이 0,1 바이너리 이거나 정수 값인 경우 해당 카테고리의 타겟 평균 값으로 인코딩 해주는 것
  + 약점 : 오버피팅이 일어날 수 있다. 타겟과 명확한 연관성을 만들어주는 것이기 때문에.
  + 규제를 추가해 노이즈 넣어주는 경우도 있다.
  
+ **frequency encoding**
  + 특정 피쳐 카테고리가 전체에서 차지하는 비중으로 인코딩

+ **label encoding**
+ **target encoding**

+ 인코딩은 다양하게 해봐야 한다. 